In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from wordcloud import WordCloud,STOPWORDS
stopwords = set(STOPWORDS)

from textblob import TextBlob
import re
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Tweeter text data in World

In [ ]:
df = pd.read_csv("../input/covid19-tweets/covid19_tweets.csv")
df.head()

## India geographical data

In [ ]:
city = pd.read_csv("../input/all-city-name-in-india/in.csv")
city.head()

### Cities in India

In [ ]:
city_name = list(city['city'])
city_name

### Extract India Twitter data using filter "city name"

In [ ]:
rslt_df = df.loc[df['user_location'].isin(city_name)].reset_index(drop=True)
rslt_df.head()

## Sort the data on the basis of date

In [ ]:
ind_tweets = rslt_df.sort_values(by='date').reset_index(drop=True)
ind_tweets

## Take the only usefull columns

In [ ]:
ind_tweets = ind_tweets[['date', 'text']]
ind_tweets.head()

### Load April data

In [ ]:
apl = pd.read_csv("../input/india-twiiter-data-in-text-format/india_tweets_data_april .csv")
apl.head()

In [ ]:
apl['date'] = apl['date'].str.replace('T',' ')
apl['date'] = apl['date'].str.replace('Z',' ')
apl.head()

In [ ]:
apl.info()

In [ ]:
ind_tweets = pd.concat([ind_tweets,apl]).reset_index(drop=True)
ind_tweets.head()

In [ ]:
ind_tweets['date'] = pd.to_datetime(ind_tweets['date'], utc=True)
ind_tweets.info()

## data preprocessing

In [ ]:
for i in range(ind_tweets.shape[0]):
    ind_tweets['text'][i] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(#[A-Za-z0-9]+)", " ", ind_tweets['text'][i]).split()).lower()
    
ind_tweets['text'].head()

## Null values

In [ ]:
ind_tweets.isnull().sum()

### WordCloud in April

In [ ]:
ap_tweet = ind_tweets[ind_tweets['date']<"2020-07-01"]['text']
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=200,
                      max_font_size=40).generate(str(ap_tweet))
fig = plt.figure(1, figsize=(15, 15))
plt.axis('off')
plt.title('cloud', size = 25)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

### WordCloud in July

In [ ]:
# tweets in 7 month
sv_tweet = ind_tweets[(ind_tweets['date']<"2020-08-01") & (ind_tweets['date']>"2020-07-01")]['text']
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=200,
                      max_font_size=40).generate(str(sv_tweet))
fig = plt.figure(1, figsize=(15, 15))
plt.axis('off')
plt.title('cloud', size = 25)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

### WordCloud in August

In [ ]:
# tweets in 8 month
ei_tweet = ind_tweets[ind_tweets['date']>="2020-08-01"]['text']
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=200,
                      max_font_size=40).generate(str(ei_tweet))
fig = plt.figure(1, figsize=(15, 15))
plt.axis('off')
plt.title('cloud', size = 25)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

### Stopwords

In [ ]:
stopwords

### Removing stopwords in the tweets data

In [ ]:
ind_tweets['text'] = ind_tweets['text'].apply(lambda tweets: ' '.join([word for word in tweets.split() if word not in stopwords]))
ind_tweets['text'].head() 

# Sentiment Analysis

In [ ]:
ind_tweets['sentiment'] = ' '
for i,tweets in enumerate(ind_tweets.text) :
    blob = TextBlob(tweets)
    if blob.sentiment.polarity > 0 :
        ind_tweets['sentiment'][i] = 'positive'
    elif blob.sentiment.polarity < 0 :
        ind_tweets['sentiment'][i] = 'negative'
    else :
        ind_tweets['sentiment'][i] = 'neutral'
        
ind_tweets.head()

#### Positive words cloud in entire Dataset

In [ ]:
pos_tweet = ind_tweets[ind_tweets['sentiment']=='positive']['text']
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=50,
                      max_font_size=40).generate(str(pos_tweet))
fig = plt.figure(1, figsize=(15, 15))
plt.axis('off')
plt.title('cloud', size = 25)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

#### Negative words cloud in entire Dataset

In [ ]:
neg_tweet = ind_tweets[ind_tweets['sentiment']=='negative']['text']
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=50,
                      max_font_size=40).generate(str(neg_tweet))
fig = plt.figure(1, figsize=(15, 15))
plt.axis('off')
plt.title('cloud', size = 25)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

#### Neutral words cloud in entire Dataset

In [ ]:
net_tweet = ind_tweets[ind_tweets['sentiment']=='neutral']['text']
wordcloud = WordCloud(background_color='black', stopwords=stopwords, max_words=50,
                      max_font_size=40).generate(str(net_tweet))
fig = plt.figure(1, figsize=(15, 15))
plt.axis('off')
plt.title('cloud', size = 25)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

### Sentiment plot in April

In [ ]:
order=["neutral","positive","negative"]
sns.countplot(ind_tweets[ind_tweets['date']<"2020-07-01"]['sentiment'], order=order)

### Sentiment plot in July

In [ ]:
sns.countplot(ind_tweets[(ind_tweets['date']<"2020-08-01") & (ind_tweets['date']>"2020-07-01")]['sentiment'], order=order)

### Sentiment plot in August

In [ ]:
sns.countplot(ind_tweets[ind_tweets['date']>="2020-08-01"]['sentiment'], order=order)

## Sentiment group analysis

In [ ]:
df = ind_tweets[['date','sentiment']].copy()
df['month'] = df['date'].dt.month
df['month'] = df['month'].map({4:'April', 7:'July', 8:'August'})
a = df.reset_index().groupby(['month', 'sentiment'])['date'].count()

In [ ]:
a = a.unstack().reindex(["April","July","August"])
a

In [ ]:
cols = ['negative','neutral','positive']
a[cols] = a[cols].div(a[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
a

In [ ]:
a.plot.bar(title='Covid19 Sentiment Analysis', ylabel="Percentage of tweets in each Month",
           figsize=(12,6), rot=0,fontsize=15)

In [ ]:
b = df.reset_index().groupby(['sentiment', 'month'])['sentiment'].count().unstack()
cols = ["April","July","August"]
b = b[cols]
b = b[cols].div(b[cols].sum(axis=0), axis=1).multiply(100)

In [ ]:
b

In [ ]:
b.plot.bar(title='Covid19 Sentiment Analysis',  ylabel="Percentage of tweets in each Month",
           figsize=(12,6), rot=0, fontsize=15)